In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Dict, List, Set

import json, os, gzip, csv, sys

In [2]:
METADATA_PATH = '/home/zxia15/data_zxia15/russian-semantics/work/metadata_russian_documents.jsonl.gz'
FULL_HATHIDATA_PATH = '/scratch4/tlippin1/data/hathi_trust/hathi_index.tsv.gz'

In [3]:
data = []
with gzip.open(METADATA_PATH, 'rt', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a pandas DataFrame
metadata_df = pd.DataFrame(data)
metadata_df.head()

,title,htid,russian text ratio,author,year,form
0,"I͡Ubileĭnyĭ sbornik Literaturnago fonda, 1859-...",mdp.39015028061110,99.376453,NaN,NaN,NaN
1,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",mdp.39015009128698,98.944547,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN
2,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",pst.000001808827,98.550891,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN
3,Poėty i poėtessy /,mdp.39015006976008,99.458652,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction
4,Poėty i poėtessy /,uc1.b3798780,99.173790,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction


In [4]:
len(metadata_df), metadata_df.iloc[0]

(89353,
 title                 I͡Ubileĭnyĭ sbornik Literaturnago fonda, 1859-...
 htid                                                 mdp.39015028061110
 russian text ratio                                            99.376453
 author                                                              NaN
 year                                                                NaN
 form                                                                NaN
 Name: 0, dtype: object)

In [9]:
unique_htids : List[str] = metadata_df['htid'].unique().tolist()
len(unique_htids), unique_htids[:5]

(89353,
 ['mdp.39015028061110',
  'mdp.39015009128698',
  'pst.000001808827',
  'mdp.39015006976008',
  'uc1.b3798780'])

In [13]:
# check the data in each line of full haiti metadata
htid_dict : Dict[str, int] = {}
csv.field_size_limit(sys.maxsize)

# important data:
#   line[0]: htid
#   line[16]: written year (or 9999 as default)
with gzip.open(FULL_HATHIDATA_PATH, 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    # headers = next(reader)  # Get the headers
    for i, line in tqdm(enumerate(reader)):
        try:
            # print(line[0])
            # print(line[16], type(line[16]))
            # break
            htid = line[0]
            if htid in unique_htids:
                year_written : int = int(line[16]) if line[16] != '9999' else -1
                htid_dict[htid] = year_written

        except Exception as e:
            print(f"Error processing line {i + 1}: {e}")
            continue

15247765it [12:37:33, 330.80it/s]

Error processing line 15247716: invalid literal for int() with base 10: 'bib'


17725860it [14:34:34, 360.95it/s]

Error processing line 17725816: invalid literal for int() with base 10: 'ru '


18309039it [15:03:27, 337.76it/s]


In [14]:
len(htid_dict)

89267

In [15]:
output_file_path = 'output.csv'

# Open the file and write the dictionary to it
with open(output_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(['Key', 'Value'])
    
    # Write data
    for key, value in htid_dict.items():
        writer.writerow([key, value])

In [16]:
metadata_df.head()

,title,htid,russian text ratio,author,year,form
0,"I͡Ubileĭnyĭ sbornik Literaturnago fonda, 1859-...",mdp.39015028061110,99.376453,NaN,NaN,NaN
1,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",mdp.39015009128698,98.944547,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN
2,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",pst.000001808827,98.550891,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN
3,Poėty i poėtessy /,mdp.39015006976008,99.458652,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction
4,Poėty i poėtessy /,uc1.b3798780,99.173790,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction


In [21]:
metadata_df['year_copywright'] = metadata_df['htid'].map(lambda x : htid_dict.get(x, -2))
metadata_df.head()

,title,htid,russian text ratio,author,year,form,year_copywright
0,"I͡Ubileĭnyĭ sbornik Literaturnago fonda, 1859-...",mdp.39015028061110,99.376453,NaN,NaN,NaN,1909
1,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",mdp.39015009128698,98.944547,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN,1909
2,"Gogolʹ; tvorchestvo, zhiznʹ i religii͡a.",pst.000001808827,98.550891,"Merezhkovsky, Dmitry Sergeyevich,",1941.0,NaN,1909
3,Poėty i poėtessy /,mdp.39015006976008,99.458652,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction,1922
4,Poėty i poėtessy /,uc1.b3798780,99.173790,"Aĭkhenvalʹd, I︠U︡. I.",1928.0,nonfiction,1922


In [22]:
len(metadata_df)

89353

In [25]:
metadata_df[(metadata_df['year_copywright'] == -2)]

,title,htid,russian text ratio,author,year,form,year_copywright
8109,Dokumenti za di͡einostʾ'ta na rusiti͡e,mdp.39015050617367,99.764175,NaN,NaN,NaN,-2
8611,Illi͡ustrirovannoe polnoe sobranie sochineniĭ /,mdp.39015005633295,99.672788,"Gogolʹ, Nikolaĭ Vasilʹevich,",1852.0,NaN,-2
11084,Gogolʹ /,mdp.39015034072176,99.221050,"Ovsi︠a︡niko-Kulikovskiĭ, D. N.",1920.0,nonfiction,-2
18776,Basni Krylova :,osu.32435016877631,99.179240,"Krylov, Ivan Andreevich,",1844.0,nonfiction,-2
19019,Uchenʹe i uchitelʹ :,uc1.$b66544,99.350784,"Pobedonost͡sev, Konstantin Petrovich,",1907.0,nonfiction,-2
...,...,...,...,...,...,...,...
82461,"Slova, izshedshīi︠a︡ izʺ ustʺ bozhīikhʺ :",nnc1.cu16181018,99.612687,"Nepli͡uev, N. N.",1908.0,nonfiction,-2
83656,"""Po prizyvu t︠s︡ari︠a︡ :",hvd.32044099662140,99.657848,"Kosovets, M.",NaN,NaN,-2
84212,Obzor Tulʹskoĭ gubernīi za ... god.,hvd.hwfte2,99.311904,NaN,NaN,NaN,-2
84574,Dogovor 11 (23) senti︠a︡bri︠a︡ 1805 goda mezhd...,hvd.hnxrub,98.986347,"Gori︠a︡inov, S. M.",1918.0,NaN,-2


In [26]:
metadata_df[(metadata_df['year_copywright'] == -1)]

,title,htid,russian text ratio,author,year,form,year_copywright
355,Trudy - Leningradskoe obshchestvo estestvoispy...,coo.31924000795223,86.258169,NaN,NaN,NaN,-1
356,Trudy - Leningradskoe obshchestvo estestvoispy...,coo.31924058781364,94.842705,NaN,NaN,NaN,-1
375,Izvi͡estīi͡a.,mdp.39015035924441,96.837903,NaN,NaN,NaN,-1
869,Li͡etopisi russkoi literatury i drevnosti...,mdp.39015008247648,98.148418,NaN,NaN,NaN,-1
1396,Matériaux inédits pour la biographie ...,mdp.39015038725449,99.290227,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...
87310,Istorii͡a chelovi͡echestva:,uiuo.ark:/13960/t9r23df8k,99.846679,NaN,NaN,nonfiction,-1
87311,Istorii͡a chelovi͡echestva:,uiuo.ark:/13960/t3rv3141w,99.787114,NaN,NaN,nonfiction,-1
87312,Istorii͡a chelovi͡echestva:,uiuo.ark:/13960/t7fr2bm09,99.621066,NaN,NaN,nonfiction,-1
88017,Polnoe sobranīe russkikh li︠e︡topiseĭ.,dul1.ark:/13960/t8fg5tc4j,99.353471,NaN,NaN,nonfiction,-1


In [27]:
DUMP_METADATA_PATH = '/home/zxia15/data_zxia15/russian-semantics/work/hathi_year_metadata_russian_documents.jsonl.gz'

# Convert DataFrame to JSON Lines and compress to .gz
with gzip.open(DUMP_METADATA_PATH, 'wt', encoding='utf-8') as f:
    for record in metadata_df.to_dict(orient='records'):
        f.write(json.dumps(record) + '\n')